### Imports

In [1]:
import monai
from monai.data import DataLoader
from monai.transforms import LoadImaged, Compose, ScaleIntensityd, RandFlipd, RandZoomd, Resized, EnsureTyped, RandRotated, ConcatItemsd, ToTensord, CenterSpatialCropd, EnsureChannelFirstd, MapTransform
from monai.metrics import ROCAUCMetric
from monai.engines import SupervisedTrainer, SupervisedEvaluator
from monai.handlers import from_engine, ValidationHandler, StatsHandler, TensorBoardStatsHandler, CheckpointSaver, TensorBoardImageHandler, ClassificationSaver, CheckpointLoader
from monai.apps import get_logger
from monai.utils import ImageMetaKey as Key
from sklearn.preprocessing import MinMaxScaler

import pandas as pd

import torch

from ignite.metrics import Accuracy

import logging

import sys

2025-02-28 07:57:06.863756: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-28 07:57:06.872281: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740729426.881027 3186401 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740729426.883609 3186401 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-28 07:57:06.894434: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

### Data loading

In [ ]:
df = pd.read_csv("data/labels_ts2024_imp.tsv", sep="\t")
df = df.assign(pet=lambda df: df['pseudo_id'].map(lambda pseudo_id: "data/cropped_nifti/" + pseudo_id + "_pet.nii.gz"))
df = df.assign(ct=lambda df: df['pseudo_id'].map(lambda pseudo_id: "data/cropped_nifti/" + pseudo_id + "_ct.nii.gz"))

df.head()

,pseudo_id,age,sex,staging,px,psa,label,alt_label,pseudo_patid,set,unknown,pet,ct
0,T_33263,NaN,M,re,0.0,0.35,1.0,0.0,96256.0,test,False,data/cropped_nifti/T_33263_pet.nii.gz,data/cropped_nifti/T_33263_ct.nii.gz
1,T_71212,NaN,M,re,1.0,8.70,1.0,1.0,28134.0,test,True,data/cropped_nifti/T_71212_pet.nii.gz,data/cropped_nifti/T_71212_ct.nii.gz
2,T_82650,NaN,M,re,1.0,0.82,1.0,1.0,75859.0,test,True,data/cropped_nifti/T_82650_pet.nii.gz,data/cropped_nifti/T_82650_ct.nii.gz
3,T_23712,NaN,M,re,1.0,932.00,0.0,0.0,20584.0,test,False,data/cropped_nifti/T_23712_pet.nii.gz,data/cropped_nifti/T_23712_ct.nii.gz
4,T_44829,NaN,M,re,0.0,3.77,1.0,1.0,28035.0,test,True,data/cropped_nifti/T_44829_pet.nii.gz,data/cropped_nifti/T_44829_ct.nii.gz


In [ ]:
df.shape

(200, 13)

In [6]:
original_df = pd.read_csv("data/labels.tsv", sep="\t")
scaler = MinMaxScaler()
scaler.fit(original_df[["psa"]])
psa_normalized = scaler.transform(df[["psa"]])
df["psa_norm"] = psa_normalized

### Label correction

In [7]:
df.loc[(df['label'] == 0) & (df['alt_label'] == 1), 'label']

150    0.0
161    0.0
172    0.0
177    0.0
Name: label, dtype: float64

In [8]:
df.loc[(df['label'] == 0) & (df['alt_label'] == 1), 'label'] = 1
df['pseudo_id'] = df['pseudo_id'].str.replace('T_', '', regex=False).astype(int)

In [9]:
df.head()

,pseudo_id,age,sex,staging,px,psa,label,alt_label,pseudo_patid,set,unknown,pet,ct,psa_norm
0,33263,NaN,M,re,0.0,0.35,1.0,0.0,96256.0,test,False,data/cropped_nifti/T_33263_pet.nii.gz,data/cropped_nifti/T_33263_ct.nii.gz,0.000047
1,71212,NaN,M,re,1.0,8.70,1.0,1.0,28134.0,test,True,data/cropped_nifti/T_71212_pet.nii.gz,data/cropped_nifti/T_71212_ct.nii.gz,0.001170
2,82650,NaN,M,re,1.0,0.82,1.0,1.0,75859.0,test,True,data/cropped_nifti/T_82650_pet.nii.gz,data/cropped_nifti/T_82650_ct.nii.gz,0.000110
3,23712,NaN,M,re,1.0,932.00,0.0,0.0,20584.0,test,False,data/cropped_nifti/T_23712_pet.nii.gz,data/cropped_nifti/T_23712_ct.nii.gz,0.125370
4,44829,NaN,M,re,0.0,3.77,1.0,1.0,28035.0,test,True,data/cropped_nifti/T_44829_pet.nii.gz,data/cropped_nifti/T_44829_ct.nii.gz,0.000507


### Create sets

In [10]:
test_data = df.to_dict('records')

### Defining the transforms

In [11]:
class Repeatd(MapTransform):

    def __init__(
        self,
        keys,
        target_size,
    ) -> None:
        MapTransform.__init__(self, keys, allow_missing_keys = True)
        self.target_size = target_size

    def __call__(self, data):

        d = dict(data)
        for key in d:
            if key in self.keys:
                d[key] = torch.Tensor([d[key]]).repeat(*self.target_size)
        return d

In [12]:
transforms = Compose(
    [
        LoadImaged(keys=["ct","pet"]),
        EnsureChannelFirstd(keys=["ct","pet"]),
        ScaleIntensityd(keys=["ct","pet"]),
        Resized(keys=["ct","pet"], spatial_size=(70, 70, 70)),
        Repeatd(keys=["psa_norm", "px"], target_size=(1, 65, 46, 69)),
        CenterSpatialCropd(keys=["ct", "pet"], roi_size = (65, 46, 69)),
        EnsureTyped(keys=["ct","pet", "psa_norm", "px"]),  
        ConcatItemsd(keys=["ct", "pet", "psa_norm", "px"], name="petct", dim=0),  
                                              
        ToTensord(keys=["petct", "ct", "pet"]),
    ]
) 

### Create data loaders

In [13]:
batchsize = 16

In [14]:
test_ds = monai.data.Dataset(data=test_data, transform=transforms)
test_loader = DataLoader(test_ds, batch_size=batchsize, num_workers=1, pin_memory=torch.cuda.is_available())

### Create model

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = monai.networks.nets.DenseNet121(spatial_dims=3, in_channels=4, out_channels=2).to(device)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 1e-5)
auc_metric = ROCAUCMetric()

### Use SupervisedTrainer

In [17]:
prepare_batch = lambda batch, device, _non_blocking: (batch["petct"].to(device), batch["label"].to(device))

##### Create handlers + Trainer and Evaluator

In [18]:
trainer = SupervisedTrainer(
    device = device,
    max_epochs = 15,
    train_data_loader = test_loader,
    network = model,
    optimizer = optimizer,
    loss_function = loss_function,
    prepare_batch = prepare_batch,
    key_train_metric = {"train_acc": Accuracy(output_transform=from_engine(["pred", "label"]))},
    amp = False
)

### Prediction

In [19]:
handler = CheckpointLoader(f"/data/f18-psma-pet-ct-ml/runs_prostate_marko_model7d/checkpoint_epoch=10.pt", load_dict={"net": model, "opt": optimizer})
handler(trainer)

/home/iimog/mambaforge/envs/f18-psma-pet-ct-ai/lib/python3.12/site-packages/monai/handlers/checkpoint_loader.py:125: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint

In [20]:
model.eval()
for batch in iter(test_loader):
    IDs = batch["pseudo_id"]
    Preds = model(batch["petct"].to(device)).argmax(dim=1)
    for ID, Pred in zip(IDs, Preds):
        df.loc[df.pseudo_id == ID.item(), 'prediction'] = Pred.item()
        print(ID, Pred)
model.train()

tensor(33263) metatensor(0, device='cuda:0')
tensor(71212) metatensor(0, device='cuda:0')
tensor(82650) metatensor(0, device='cuda:0')
tensor(23712) metatensor(0, device='cuda:0')
tensor(44829) metatensor(1, device='cuda:0')
tensor(89795) metatensor(0, device='cuda:0')
tensor(43412) metatensor(0, device='cuda:0')
tensor(86419) metatensor(0, device='cuda:0')
tensor(28330) metatensor(0, device='cuda:0')
tensor(75117) metatensor(1, device='cuda:0')
tensor(2513) metatensor(0, device='cuda:0')
tensor(41025) metatensor(0, device='cuda:0')
tensor(24242) metatensor(0, device='cuda:0')
tensor(15014) metatensor(1, device='cuda:0')
tensor(86015) metatensor(0, device='cuda:0')
tensor(61820) metatensor(0, device='cuda:0')
tensor(62629) metatensor(0, device='cuda:0')
tensor(90149) metatensor(0, device='cuda:0')
tensor(8645) metatensor(0, device='cuda:0')
tensor(6614) metatensor(0, device='cuda:0')
tensor(86271) metatensor(1, device='cuda:0')
tensor(53582) metatensor(0, device='cuda:0')
tensor(56253)

DenseNet121(
  (features): Sequential(
    (conv0): Conv3d(4, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
    (norm0): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (layers): Sequential(
          (norm1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv3d(64, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (norm2): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv3d(128, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        )
      )
      (denselayer2): _DenseLayer(
        (layers): Sequential(
 

In [ ]:
df.to_csv(path_or_buf="analysis/testset_predictions.tsv", sep="\t", index=False)

In [23]:
df.value_counts(["label", "prediction"])

label  prediction
0.0    0.0           77
1.0    1.0           60
       0.0           51
0.0    1.0           12
Name: count, dtype: int64

In [ ]:
from sklearn.metrics import balanced_accuracy_score, accuracy_score

In [28]:
accuracy_score(df.label, df.prediction)

0.685

In [29]:
balanced_accuracy_score(df.label, df.prediction)

np.float64(0.7028545399331916)

In [30]:
specificity = 77/89
sensitivity = 60/111
print(f"specificity = {specificity}")
print(f"sensitivity = {sensitivity}")

specificity = 0.8651685393258427
sensitivity = 0.5405405405405406


In [34]:
dfu = df[df.unknown]
dfk = df[~df.unknown]

In [35]:
dfu.shape, dfk.shape

((116, 15), (84, 15))

In [36]:
accuracy_score(dfu.label, dfu.prediction)

0.7327586206896551

In [37]:
accuracy_score(dfk.label, dfk.prediction)

0.6190476190476191